# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

SyntaxError: invalid syntax (<ipython-input-3-71a7702f86c3>, line 3)

In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

SyntaxError: invalid syntax (<ipython-input-4-6645883cfe43>, line 3)

****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot() 

# Anwers

EXERCISE 1 

In [6]:
import pandas as pd

In [7]:
# Extracting values to future DataFrame - PLan: Create a easy dataframe and use Pandas on it



ex1 = { 'mort': [], 'name':[]}


for x in root.iter('country'):
    for name,mort in zip(x.findall('name'),x.findall('infant_mortality')):
        ex1['name'].append(name.text)
        ex1['mort'].append(mort.text)
    
    
# Using Pandas to show the results

data = pd.DataFrame(data=ex1)
data = data[['name','mort']]
data['mort']=data.mort.astype(float)

data.sort_values(by='mort').head(10)


,name,mort
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


EXERCISE 2

In [25]:
import numpy as np

In [170]:
# Extracting values to future DataFrame - PLan: Create a easy dataframe and use Pandas on it

#ASSUMING THE POPULATION ONLY IN CITIES WITH 2011 CENSUS INFO - NOT DESCRIBED IN THE QUESTION

ex2 = { 'pop': [], 'name':[]}

for ct in root.iter('city'):
    ex2['name'].append(ct[0].text)
    #checking if there is a population tag
    p = ct.find('population')
    if p is None:
        #if not, nan
        ex2['pop'].append(np.nan)
    else:
        # if there is:
        #iterate into population tags to check if is there a attrib = 2011
        check=[]
        for x in ct.iter('population'):
            check.append(x.attrib['year'])
        if '2011' in check:
            for x in ct.iter('population'):
                if x.attrib['year'] == '2011':
                    ex2['pop'].append(x.text)
        else:
            ex2['pop'].append(np.nan)
            
          
    
    
# Using Pandas to show the results

data = pd.DataFrame(data=ex2)
data_clean= data.dropna(axis=0)
data_clean['pop'] = data_clean['pop'].astype(int)



data_clean.sort_values(by='pop', ascending=False).head(10)


F:\Anaconda2\lib\site-packages\ipykernel\__main__.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,name,pop
1527,Mumbai,12442373
1582,Delhi,11034555
1515,Bangalore,8443675
1000,London,8250205
1382,Tehran,8154051
1470,Dhaka,7423137
1591,Hyderabad,6731790
1505,Ahmadabad,5577940
3056,Luanda,5000000
1556,Chennai,4646732


EXERCISE 3

In [226]:
# Extracting values to future DataFrame - PLan: Create a easy dataframe and use Pandas on it

ex3 = {}

for ct in root.iter('country'):
    #checking if there is a ethnicgroup tag
    p = ct.find('ethnicgroup')
    if p is not None:
    # Final Plan:  ex3['ethnicgroup'] += % * population
        
        #getting population variable
        bestyear = []
        for pop in ct.findall('population'):
            bestyear.append(pop.attrib['year'])
        
        population= []
        for pop in ct.findall('population'):
            if pop.attrib['year'] == bestyear[len(bestyear) -1]:
                population.append(pop.text)
        
    
        for et in ct.findall('ethnicgroup'):
            a = et.get('percentage')
            ex3[et.text] =+ ((float(a)/100 )*(float(population[0])))

# seting dict to pandas dataframe            
ex33 = {'ethnic': [], 'pop' : []}
for etc,pop in ex3.items():
    ex33['ethnic'].append(etc)
    ex33['pop'].append(pop)   
    
# Using Pandas to show the results
data = pd.DataFrame(data=ex33)
data.sort_values(by='pop', ascending=False).head(10)



,ethnic,pop
80,Han Chinese,1.245059e+09
106,Indo-Aryan,8.718156e+08
105,Dravidian,3.027137e+08
98,Bengali,1.467769e+08
139,Japanese,1.265342e+08
130,Eastern Hamitic,8.283038e+07
147,Mulatto,7.806590e+07
119,Viet/Kinh,7.607838e+07
70,English,5.359233e+07
17,Mediterranean Nordic,4.681592e+07
